In [ ]:
#from: https://github.com/iShkiper/DSP_24.M20_21/blob/main/%D0%9A%D0%BE%D0%B4/1.1.0%20DFT_1D_on.ipynb

In [5]:
from scipy.fftpack import fft, ifft, fftshift
from scipy.io import wavfile
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import get_window

fs, sig = wavfile.read('source\signals\phoneNumber'+str(0)+'.wav')

In [6]:
# DTMF частоты
ROW_FREQS = [697, 770, 852, 941]
COL_FREQS = [1209, 1336, 1477]
DTMF_MAP = {
    (697, 1209): '1', (697, 1336): '2', (697, 1477): '3',
    (770, 1209): '4', (770, 1336): '5', (770, 1477): '6',
    (852, 1209): '7', (852, 1336): '8', (852, 1477): '9',
    (941, 1209): '*', (941, 1336): '0', (941, 1477): '#'
}


In [10]:
def freq_from_fft(signal, rate, freqs):  # детект частот
    N = len(signal)
    windowed = signal * get_window('hann', N)
    spectrum = np.abs(np.fft.rfft(windowed))
    freqs_bins = np.fft.rfftfreq(N, 1 / rate)
    
    detected = {}
    for target in freqs:
        idx = np.argmin(np.abs(freqs_bins - target))
        detected[target] = spectrum[idx]
    return detected

In [22]:
freq_from_fft(sig,fs,ROW_FREQS)

{697: np.float64(286.80949291999747),
 770: np.float64(5.430858378844015),
 852: np.float64(873.9289535699147),
 941: np.float64(17.317725975926216)}

In [111]:
def decode(fs,sig):
    window_size = int(0.05 * fs)
    #window_size = int(fs * 4000/11 / 1000)
    step = window_size
    digits = []

    for i in range(0, len(sig) - window_size, step):
        frame = sig[i:i+window_size]
        low_mags = freq_from_fft(frame, fs, ROW_FREQS)
        high_mags = freq_from_fft(frame, fs, COL_FREQS)

        low = max(low_mags, key=low_mags.get)
        high = max(high_mags, key=high_mags.get)
        threshold = 25
        if low_mags[low] > threshold and high_mags[high] > threshold:
            digit = DTMF_MAP.get((low, high))
            
            if digit and (not digits or digits[-1] != digit):
                digits.append(digit)
        
            
    
    return ''.join(digits)

In [112]:
decode(fs,sig)


'891750168'

In [113]:
for i in range(10):
    fs, sig = wavfile.read('source\signals\phoneNumber'+str(i)+'.wav')
    
   #### декодировать сигнал sig ####
    print(decode(fs,sig))

8921989797
89215371248
923083
8905780
8902180
890125942
8915079315
891289327
891831903
891750168
